In [1]:
import asyncio
from web3 import Web3
import os
import json
from copy import deepcopy
import time
from transactions import safe_build_and_send_tx
import config
from mach_client import client

In [4]:
def load_abi(envvar: str, default_path: str) -> dict:
    """
    Load a JSON abi from a file.

    :param envvar: Environment variable pointing to the path.
    :param default_path: The path to check if environment var isn't set.
    """
    abi_path = os.environ.get(envvar, default_path)
    with open(abi_path) as f:
        return json.load(f)
    
def load_config() -> "dict[str, any]":
    """
    Load config from ./config/deployments.json.

    Load private key.

    Load CCTP, order_book, and ERC20 ABIs.
    """

    configuration: dict = {}

    # Retrieve the deployments JSON from the FS.
    deployments_path = os.environ.get("CONFIG_PATH", "../../abi/aave.json")
    with open(deployments_path) as f:
        deployments = json.load(f)

    configuration["safe_config"] = deepcopy(deployments)

    # Add the RPC urls to the deployments JSON object without writing to FS.
    # NOTE: The list cast is to avoid modifying the original dict while iterating
    load_dotenv('.envrc')
    for deployment in list(deployments["deployments"].keys()):
        assert isinstance(deployment, str), "Configuration file is broken"
        rpc_url = os.getenv(f"{deployment.upper()}_RPC_URL")
        assert (
            rpc_url is not None
        ), f"Must supply {deployment} rpc url as an environment variable"

    configuration["deployments"] = deployments["deployments"]
    configuration["private_key"] = os.environ["PRIV_KEY"]
    configuration["order_book_abi"] = load_abi(
        "ORDER_BOOK_API_PATH", "./config/abi/order_book_abi.json"
    )
    configuration['backend_api_url'] = os.environ['BACKEND_URL']
    configuration["erc20_abi"] = load_abi(
        "ERC20_ABI_PATH", "./config/abi/erc20_abi.json"
    )
    configuration["aave_abi"] = load_abi(
        "AAVE_ABI_PATH", "./config/abi/aave_abi.json"
    )
    return configuration

In [48]:
chain_name='avalanche'
chain_info = client.deployments["avalanche"]

if not chain_info:
    raise ValueError(f"Chain {chain_name} not found in configuration")

rpc_url = client.rpc_urls[chain_name]
if not rpc_url:
    raise ValueError(f"RPC URL not found for {chain_name}")

w3 = Web3(Web3.HTTPProvider(rpc_url))

assert w3.is_connected()

def get_token_balance_in_aave(w3, pool_address, token_address, user_address):
    pool_contract = w3.eth.contract(address=pool_address, abi=CONFIG["aave_abi"])
    token_contract = w3.eth.contract(address=token_address, abi=CONFIG['erc20_abi'])
    try:
        account_data = pool_contract.functions.getUserAccountData(user_address).call()
        total_collateral_base = account_data[0]
        token_decimals = token_contract.functions.decimals().call()
        balance = total_collateral_base / (10 ** token_decimals)
        return balance
    except Exception as e:
        print(f"Error getting balance: {str(e)}")
        return 0
    
get_token_balance_in_aave(w3, "0x794a61358D6845594F94dc1DB02A252b5b4814aD", "0xB97EF9Ef8734C71904D8002F8b6Bc66Dd9c48a6E", "0xb20094DFede30AbEe3a8d549BbA828b6fd771106")


140.422808

In [41]:
balance

In [36]:
##STEP 1
CONFIG=load_config()

def get_token_balance_in_aave(w3, pool_address, token_address, user_address):
    pool_contract = w3.eth.contract(address=pool_address, abi=CONFIG["aave_abi"])
    token_contract = w3.eth.contract(address=token_address, abi=CONFIG['erc20_abi'])

    try:
        account_data = pool_contract.functions.getUserAccountData(token_address, user_address).call()
        total_collateral_base = account_data[0]
        token_decimals = token_contract.functions.decimals().call()
        balance = total_collateral_base / (10 ** token_decimals)
        return balance
    except Exception as e:
        print(f"Error getting balance: {str(e)}")
        return 0

def get_highest_balance_on_chain(user_address, chain_name):
    chain_info = CONFIG["safe_config"]["deployments"].get(chain_name)
    if not chain_info:
        raise ValueError(f"Chain {chain_name} not found in configuration")

    rpc_url = os.getenv(f"{chain_name.upper()}_RPC_URL")
    if not rpc_url:
        raise ValueError(f"RPC URL not found for {chain_name}")

    w3 = Web3(Web3.HTTPProvider(rpc_url))
    if not w3.is_connected():
        raise ConnectionError(f"Unable to connect to {chain_name} network")

    aave_market_address = chain_info["contracts"]["aave_market"]
    highest_balance = 0
    highest_asset = None
    non_zero_assets = []

    for asset_name, asset_info in chain_info["assets"].items():
        balance = get_token_balance_in_aave(w3, aave_market_address, asset_info["address"], user_address)
        if balance > 0:
            non_zero_assets.append(asset_name)
            if balance > highest_balance:
                highest_balance = balance
                highest_asset = asset_name

    if len(non_zero_assets) > 1:
        raise MultipleNonZeroBalancesError(f"Multiple non-zero balances found: {', '.join(non_zero_assets)}")

    return highest_asset, highest_balance

def main():
    user_address = "0x742d35Cc6634C0532925a3b844Bc454e4438f44e"  # Replace with the actual user address
    chain_name = "avalanche"  # Replace with the desired chain name

    try:
        asset, balance = get_highest_balance_on_chain(user_address, chain_name)
        if asset:
            print(f"Highest Aave balance on {chain_name} for user {user_address}:")
            print(f"  {asset}: {balance:.4f}")
        else:
            print(f"No Aave balances found on {chain_name} for user {user_address}")
    except MultipleNonZeroBalancesError as e:
        print(f"Error: {str(e)}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [38]:
chain_name='arbitrum'

chain_info = CONFIG["safe_config"]["deployments"].get(chain_name)

rpc_url = os.getenv(f"{chain_name.upper()}_RPC_URL")
if not rpc_url:
    raise ValueError(f"RPC URL not found for {chain_name}")

w3 = Web3(Web3.HTTPProvider(rpc_url))
if not w3.is_connected():
    raise ConnectionError(f"Unable to connect to {chain_name} network")

aave_market_address = chain_info["contracts"]["aave_market"]
highest_balance = 0
highest_asset = None
non_zero_assets = []

chain_info["assets"].items()

Error getting balance: 
Could not identify the intended function with name `getUserAccountData`, positional arguments with type(s) `address,address` and keyword arguments with type(s) `{}`.
Found 1 function(s) with the name `getUserAccountData`: ['getUserAccountData(address)']
Function invocation failed due to improper number of arguments.


dict_items([('USDC', {'address': '0xaf88d065e77c8cC2239327C5EDb3A432268e5831', 'decimals': 6}), ('USDT', {'address': '0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9', 'decimals': 6}), ('DAI', {'address': '0xDA10009cBd5D07dd0CeCc66161FC93D7c9000da1', 'decimals': 18}), ('crvUSD', {'address': '0x498Bf2B1e120FeD3ad3D42EA2165E9b73f99C1e5', 'decimals': 18}), ('FRAX', {'address': '0x17FC002b466eEc40DaE837Fc4bE5c67993ddBd6F', 'decimals': 18})])

In [39]:
aave_market_address

'0x794a61358D6845594F94dc1DB02A252b5b4814aD'

In [20]:
##STEP 2 get the highest rate 
def get_market_data(w3: Web3, pool_contract, asset_address):
    reserve_data = pool_contract.functions.getReserveData(asset_address).call()
    scaling_factor = 10**27
    return {
        "liquidity_rate": reserve_data[2] / scaling_factor,
        "variable_borrow_rate": reserve_data[4] / scaling_factor,
        "stable_borrow_rate": reserve_data[5] / scaling_factor,
    }

def get_highest_liquidity_rate_asset(current_chain):
    highest_rate = -1
    highest_asset = None
    highest_chain = None

    for chain_name, chain_info in CONFIG["safe_config"]["deployments"].items():
        if chain_name == current_chain:
            continue  # Skip the current chain

        rpc_url = os.getenv(f"{chain_name.upper()}_RPC_URL")
        if not rpc_url:
            print(f"  Error: RPC URL not found for {chain_name}")
            continue

        w3 = Web3(Web3.HTTPProvider(rpc_url))
        if not w3.is_connected():
            print(f"  Error: Unable to connect to {chain_name} network")
            continue

        aave_market_address = chain_info["contracts"]["aave_market"]
        pool_contract = w3.eth.contract(address=aave_market_address, abi=CONFIG['aave_abi'])

        for asset_name, asset_info in chain_info['assets'].items():
            try:
                market_data = get_market_data(w3, pool_contract, asset_info['address'])
                liquidity_rate = market_data['liquidity_rate']
                if liquidity_rate > highest_rate:
                    highest_rate = liquidity_rate
                    highest_asset = asset_name
                    highest_chain = chain_name
                
            except Exception as e:
                print(f"  Error fetching data for {asset_name} on {chain_name}: {str(e)}")

    print(f"Highest liquidity rate asset not on {current_chain}:")
    print(f"  Asset: {highest_asset}")
    print(f"  Chain: {highest_chain}")
    print(f"  Liquidity Rate: {highest_rate:.2%}")

    return highest_asset, highest_rate, highest_chain

##example
current_chain='avalanche'
get_highest_liquidity_rate_asset(current_chain)

Highest liquidity rate asset not on avalanche:
  Asset: USDT
  Chain: arbitrum
  Liquidity Rate: 3.64%


('USDT', 0.03644915859872191, 'arbitrum')

Highest liquidity rate asset not on avalanche:
  Asset: USDT
  Chain: arbitrum
  Liquidity Rate: 3.64%


('USDT', 0.03643517427740931, 'arbitrum')

In [4]:
CONFIG['aave_abi']

[{'inputs': [{'internalType': 'contract IPoolAddressesProvider',
    'name': 'provider',
    'type': 'address'}],
  'stateMutability': 'nonpayable',
  'type': 'constructor'},
 {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'reserve',
    'type': 'address'},
   {'indexed': True,
    'internalType': 'address',
    'name': 'backer',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'amount',
    'type': 'uint256'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'fee',
    'type': 'uint256'}],
  'name': 'BackUnbacked',
  'type': 'event'},
 {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'reserve',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'address',
    'name': 'user',
    'type': 'address'},
   {'indexed': True,
    'internalType': 'address',
    'name': 'onBehalfOf',
    'type': 'address'},
   {'indexed': False,
  

In [5]:
CONFIG=load_config()
for chain_name, chain_info in CONFIG["safe_config"]["deployments"].items():
    print(f"\n{chain_name}:")
    print(f"Chain Info: {chain_info}")
    
    aave_market_address = chain_info["contracts"]["aave_market"]
    print(f"Aave Market Address: {aave_market_address}")
    
    rpc_url = os.getenv(f"{chain_name.upper()}_RPC_URL")
    print(f"RPC URL: {rpc_url}")
    
    if not rpc_url:
        print(f"  Error: RPC URL not found for {chain_name}")
        continue

    w3 = Web3(Web3.HTTPProvider(rpc_url))
    
    if not w3.is_connected():
        print(f"  Error: Unable to connect to {chain_name} network")
        continue
    
    print(f"Current Block Number: {w3.eth.block_number}")

    pool_contract = w3.eth.contract(address=aave_market_address, abi=CONFIG['aave_abi'])

    # Assuming the tokens are still defined in the chain_info
    for asset_name, asset_info in chain_info['assets'].items():
        try:
            market_data = get_market_data(w3, pool_contract, asset_info['address'])
            print(f"  {asset_name}:")
            print(f"    Liquidity Rate: {market_data['liquidity_rate']:.2%}")
            print(f"    Variable Borrow Rate: {market_data['variable_borrow_rate']:.2%}")
        except Exception as e:
            print(f"  Error fetching data for {asset_name}: {str(e)}")
    else:
        print("  No tokens defined for this chain")



arbitrum:
Chain Info: {'chain_id': 42161, 'abbreviation': 'ARB', 'lz_cid': 110, 'cctp_id': 3, 'evm_block_time': 12, 'contracts': {'order_book': '0xa40Ad3916237fa0FE11A500241fFA6eAc59CBD6A', 'aave_market': '0x794a61358D6845594F94dc1DB02A252b5b4814aD', 'cctp_message_transmitter': '0xC30362313FBBA5cf9163F0bb16a0e01f01A896ca', 'cctp_token_messenger': '0x19330d10D9Cc8751218eaf51E8885D058642E08A', 'uniswap_v3_quoter': '0x61fFE014bA17989E743c5F6cB21bF9697530B21e', 'uniswap_v3_router': '0x5E325eDA8064b456f4781070C0738d849c824258', 'uniswap_permit2': '0x000000000022D473030F116dDEE9F6B43aC78BA3', 'curve_router': '0xF0d4c12A5768D806021F80a262B4d39d26C58b8D'}, 'assets': {'ETH': {'address': '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1', 'decimals': 18, 'usd_est': 300000000000000.0, 'wrapped': True, 'symbol_override': True}, 'USDC': {'address': '0xaf88d065e77c8cC2239327C5EDb3A432268e5831', 'decimals': 6}, 'USDT': {'address': '0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9', 'decimals': 6}, 'DAI': {'addr

In [6]:

for asset_name, asset_info in chain_info['assets'].items():
    print(asset_name, asset_info)

    market_data = get_market_data(w3, pool_contract, asset_info['address'], asset_info['decimals'])
    print(asset_name, asset_info['decimals'], market_data)

USDC {'address': '0xB97EF9Ef8734C71904D8002F8b6Bc66Dd9c48a6E', 'decimals': 6}


TypeError: get_market_data() takes 3 positional arguments but 4 were given

In [ ]:
market_data

{'liquidity_rate': 0.026077989108583707,
 'variable_borrow_rate': 0.04609634678177154,
 'stable_borrow_rate': 0.0}

In [ ]:
async def get_market_data(w3, pool_contract, token_address):
    reserve_data = await pool_contract.functions.getReserveData(token_address).call()
    return {
        "liquidity_rate": reserve_data[2] / 1e27,
        "variable_borrow_rate": reserve_data[4] / 1e27,
        "stable_borrow_rate": reserve_data[5] / 1e27,
    }

In [ ]:
async def main():
    for chain_name, chain_info in CHAINS.items():
        print(f"\n{chain_name} Chain:")
        w3 = Web3(AsyncHTTPProvider(chain_info["rpc_url"]), modules={"eth": (AsyncEth,)})
        pool_contract = w3.eth.contract(address=chain_info["pool_address"], abi=AAVE_V3_POOL_ABI)

        for token_name, token_address in TOKENS.items():
            try:
                market_data = await get_market_data(w3, pool_contract, token_address)
                print(f"  {token_name}:")
                print(f"    Liquidity Rate: {market_data['liquidity_rate']:.2%}")
                print(f"    Variable Borrow Rate: {market_data['variable_borrow_rate']:.2%}")
                print(f"    Stable Borrow Rate: {market_data['stable_borrow_rate']:.2%}")
            except Exception as e:
                print(f"  Error fetching data for {token_name}: {str(e)}")

if __name__ == "__main__":
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop